Install the requirements

In [ ]:
!pip install datasets evaluate rouge_score
!pip install ffmpeg
!pip install git+https://github.com/openai/whisper.git
!pip install -U --no-cache-dir accelerate transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=4f7ef5920932f1b264d36c7d37ec8dbaddb3b0567e747aacd3bcb0ad555f1a9e
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6082 sha256=32d834043c79e120bfb7fc2b74d2aa1493958d47e970c09b4aada0a6ec62b221
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg
  Cloning h

In [ ]:
import subprocess
import whisper
command = 'ffmpeg -i /content/aiml.mkv -ab 160k -ar 44100 -vn /content/audio.wav'
subprocess.call(command, shell=True)

0

In [ ]:
model = whisper.load_model('base')

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 104MiB/s]


In [ ]:
res = model.transcribe('/content/audio.wav')

In [ ]:
res['text']

' The inflation reduction act lowers prescription drug costs, health care costs and energy costs. It is the most aggressive action on tackling the climate crisis in American history, which will lift off American workers and create good paying union jobs across the country. It will lower the deficit and ask the ultra-veldian corporations to pay their fair share and no one making under $400,000 per year will pay up any more in taxes.'

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Import the dataset


In [ ]:
from datasets import load_dataset
billsum = load_dataset("billsum", split="ca_test")


Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

split the datset

In [ ]:
billsum = billsum.train_test_split(test_size=0.2)

example check

In [ ]:
billsum["train"][0]

{'text': 'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 14005.7 of the Welfare and Institutions Code is amended to read:\n14005.7.\n(a) Medically needy persons and medically needy family persons are entitled to health care services under Section 14005 providing all eligibility criteria established pursuant to this chapter are met.\n(b) Except as otherwise provided in this chapter or in Title XIX of the federal Social Security Act, no medically needy family person, medically needy person or state-only Medi-Cal persons shall be entitled to receive health care services pursuant to Section 14005 during any month in which his or her share of cost has not been met.\n(c) In the case of a medically needy person, monthly income, as determined, defined, counted, and valued, in accordance with Title XIX of the federal Social Security Act, in excess of the amount required for maintenance established pursuant to Section 14005.12, exclusive of any amounts consid

loading a T5 tokenizer to process text and summary:

In [ ]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

create the preprocissing function to tell the t5 model to do summarisation task and truncate the summary at a certain number of tokens to preserve performance and increase acc

In [ ]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=100, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

creating a batch of examples using datacollatorseq2seq. since it is more efficient to pad the examples in batches

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

evaluate the model using the rouge metric

In [ ]:
import evaluate
import numpy as np

rouge = evaluate.load("rouge")



def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

Train the model and tune the hyperparameters

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
!pip install accelerate -U
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

training_args = Seq2SeqTrainingArguments(
    output_dir="CSE4022_NLP_EPJ_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=20,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.608489,0.145200,0.046300,0.119800,0.119300,19.000000
2,No log,2.470221,0.168300,0.059000,0.137300,0.137200,19.000000
3,No log,2.392994,0.198200,0.082400,0.163600,0.163500,19.000000
4,No log,2.349102,0.222100,0.103600,0.185600,0.185600,19.000000
5,No log,2.320213,0.228100,0.108100,0.190300,0.190500,19.000000
6,No log,2.293399,0.225500,0.107200,0.189100,0.189200,19.000000
7,No log,2.272245,0.227200,0.111000,0.191300,0.191300,19.000000
8,No log,2.251587,0.228400,0.111900,0.192200,0.192400,19.000000
9,2.598000,2.238634,0.230100,0.113300,0.194000,0.194200,19.000000
10,2.598000,2.227638,0.229000,0.113400,0.193600,0.193800,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control

TrainOutput(global_step=1240, training_loss=2.440059834141885, metrics={'train_runtime': 1628.5738, 'train_samples_per_second': 12.146, 'train_steps_per_second': 0.761, 'total_flos': 5354121666232320.0, 'train_loss': 2.440059834141885, 'epoch': 20.0})

In [ ]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1700846566.e482e1ff18de.2978.0:   0%|          | 0.00/16.4k [00:00<?, ?B/s]

'https://huggingface.co/Kitteshwar/CSE4022_NLP_EPJ_model/tree/main/'

using the model

In [ ]:
text1 = "summarize: "
text2= res['text']
text=text1+text2
from transformers import pipeline

summarizer = pipeline("summarization", model="Kitteshwar/CSE4022_NLP_EPJ_model",max_length=50)
summarizer(text)

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

[{'summary_text': 'the inflation reduction act lowers prescription drug costs, health care costs and energy costs. It is the most aggressive action on tackling the climate crisis in American history, which will lift off workers and create good paying union jobs across the country.'}]